# Reconstructing the genome from a convolutional neural network

In this notebook, I will go through using the weights and activations to reconstruct the genome using decorrelation and unpooling.  


In [ ]:
from __future__ import print_function
import os, sys, urllib, gzip, matplotlib
sys.setrecursionlimit(10000)
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib as mpl
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})

%matplotlib inline
from scipy.misc import imresize

from six.moves import cPickle
import h5py

import numpy as np
from IPython.display import Image as IPImage
from PIL import Image
import cPickle as pickle

from lasagne import layers
from lasagne.layers import get_output, get_output_shape, get_all_layers, get_all_params
from lasagne.layers import get_all_param_values, set_all_param_values, get_all_layers
from lasagne import nonlinearities
from lasagne import updates 
from lasagne import objectives 
from lasagne import init 
import theano.tensor as T
import theano

sys.path.append('..')
from src import NeuralNet
from models import load_model
from data import load_data

In [ ]:
# load data
name = 'MotifSimulation_correlated'
datapath = '/home/peter/Data/SequenceMotif'
filepath = os.path.join(datapath, 'synthetic_correlated_motifs_300000.hdf5')
train, valid, test = load_data(name, filepath)
shape = (None, train[0].shape[1], train[0].shape[2], train[0].shape[3])
num_labels = np.round(train[1].shape[1])

In [ ]:
# load model
model_name = "binary_genome_motif_model"
nnmodel = NeuralNet(model_name, shape, num_labels)

In [ ]:
# set the parameters of the model
savepath = '/home/peter/Data/SequenceMotif/Results/binary_epoch_14.pickle'
nnmodel.set_parameters_from_file(savepath)
network = nnmodel.network

In [ ]:
# get the weights of each layer in the network
all_layers = get_all_layers(network['output_bias'])
W = []
b = []
active_index = []
for index in range(len(all_layers)):
    layer = all_layers[index]
    output_shape = layer.output_shape
    params = layer.params
    if params:
        keys = params.keys()
        for j in range(len(keys)):
            if 'b' == str(keys[j]): 
                b.append(layer.b.get_value())
            if 'W' in str(keys[j]):
                W.append(layer.W.get_value())
            if 'alpha' == str(keys[j]):
                active_index.append(index)

In [ ]:
# get feature maps of a given layer
layer = all_layers[active_index[0]]
map_shape = get_output_shape(layer)
input_var = T.tensor4('input')
fmap = get_output(all_layers[1])
feature_maps = theano.function([input_var], [fmap], allow_input_downcast=True)

In [ ]:
#get_feature_maps(train[0][0]) 

map_shape